In [4]:
%load_ext autoreload

In [5]:
%autoreload 1

In [96]:
%aimport src.lsh
%aimport settings
%aimport src.hashes
%aimport src.utils
%aimport src.lsh_knn

In [38]:
import os
import sys
sys.path.append(os.environ.get("ALGO_COURSE_WORK_REPO"))

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [169]:
from src.hashes import MinHash
from src.utils import bytes_to_crc_32_int, bytes_to_sha_256_int
from src.lsh import MinHashLSH

In [168]:
data = ["hello world. I'm here for you.",
        "how are you?",
        "I don't give a fuck."
       ]
minhashes = {}
for k, sentence in enumerate(data):
    mh = MinHash(num_permutations=128, seed=10, hash_function=bytes_to_crc_32_int)
    for token in sentence.split():
        mh.update(token.encode("utf-8"))
    minhashes[k] = mh
        
lsh = MinHashLSH(num_permutations=128, params=(10, 12))
lsh.batch_add(minhashes)
lsh.index()

# Query
query = "hello world."
mh = MinHash(num_permutations=128, seed=10, hash_function=bytes_to_crc_32_int)
for token in query.split():
    mh.update(token.encode("utf-8"))
idx = lsh.query(mh, k=1)[0]
print(f"Best neighbour: `{data[idx]}`")

Best neighbour: `hello world. I'm here for you.`
